# Node classification for Heterogeneous graph

## 0. Enviroment setup

In [1]:
# !pip uninstall torch torchvision torchaudio --yes
# !pip install torch==2.2.1 torchvision==0.17.1 torchaudio==2.2.1 --index-url https://download.pytorch.org/whl/cu121
# !pip install lightning torch_geometric
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.2.0+cu121.html
# !pip install wandb

In [2]:
import os
import shutil
import wandb
import torch
import torch.nn as nn
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from tqdm import tqdm

from torch_geometric.utils import negative_sampling
import torch_geometric.transforms as T
from torch_geometric.utils import train_test_split_edges
# from torch_geometric.loader import LinkNeighborLoader
# from torch_geometric.data.lightning import LightningLinkData

import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint
from lightning.pytorch.loggers import WandbLogger

In [3]:
from hive_analysis.models.homo_node_classification import *
from hive_analysis.dataloaders import hive_preprocessing

In [4]:
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

In [5]:
# Setup device agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## 1. Data Pre-processing

In [6]:
DATA_VERSION = 'final_v1'

In [7]:
# data = hive_preprocessing(
#     f'dataset/hive/{DATA_VERSION}/nodes_labelled.csv',
#     f'dataset/hive/{DATA_VERSION}/edges_labelled.csv',
#     to_undirected = True,
# )
# torch.save(data, f'dataset/hive/{DATA_VERSION}/hive.pt')
data = torch.load(f'dataset/hive/{DATA_VERSION}/hive.pt')
data

HeteroData(
  user={
    x=[18645, 5],
    node_label=[18645],
  },
  comment={
    x=[125111, 1],
    node_label=[125111],
  },
  post={
    x=[13540, 1],
    node_label=[13540],
  },
  (user, upvote, comment)={
    edge_index=[2, 423638],
    edge_attr=[423638, 1],
    y=[423638],
  },
  (user, upvote, post)={
    edge_index=[2, 554131],
    edge_attr=[554131, 1],
    y=[554131],
  },
  (user, write, comment)={
    edge_index=[2, 78696],
    edge_attr=[78696, 1],
    y=[78696],
  },
  (user, write, post)={
    edge_index=[2, 12958],
    edge_attr=[12958, 1],
    y=[12958],
  },
  (user, downvote, comment)={
    edge_index=[2, 6819],
    edge_attr=[6819, 1],
    y=[6819],
  },
  (user, downvote, post)={
    edge_index=[2, 2934],
    edge_attr=[2934, 1],
    y=[2934],
  },
  (comment, belong_to, comment)={
    edge_index=[2, 117822],
    edge_attr=[117822, 1],
    y=[117822],
  },
  (comment, belong_to, post)={
    edge_index=[2, 19838],
    edge_attr=[19838, 1],
    y=[19838],
  },
  

In [8]:
homo_data = data.to_homogeneous()

In [9]:
transform = T.RandomNodeSplit(
    num_val=0.1,
    num_test=0.1,
    key='node_label',
)

homo_data = transform(homo_data)

In [10]:
homo_data

Data(edge_index=[2, 2315850], x=[157296, 5], node_label=[157296], edge_attr=[2315850, 1], y=[2315850], node_type=[157296], edge_type=[2315850], train_mask=[157296], val_mask=[157296], test_mask=[157296])

## 2. Training

In [11]:
models = {
    'GraphConv': GraphConvNet, 
    'GATv2': GATv2, 
    'GraphSAGE': GraphSAGE, 
    # 'GAT':GAT
}

In [12]:
models = { k: m(
    in_channels=-1,  
    hidden_channels=[64, 128, 128, 256, 256], 
    # aggr_scheme='mean',
) for k, m in models.items()}

In [13]:
class GraphDataset(Dataset):
    def __init__(
        self,
        data,
        mask_key,
    ):
        self.data = data
        self.train_mask = data[mask_key]

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        return self.data, self.train_mask
    
def collate_fn(input):
    data, mask = zip(*input)
    return data[0], mask[0]

train_loader = DataLoader(
    GraphDataset(homo_data, 'train_mask'),
    batch_size=1,
    shuffle=True,
    drop_last=False,
    pin_memory=True,
    num_workers=4,
    collate_fn=collate_fn,
)
val_loader = DataLoader(
    GraphDataset(homo_data, 'val_mask'),
    batch_size=1,
    shuffle=False,
    drop_last=False,
    pin_memory=True,
    num_workers=4,
    collate_fn=collate_fn
)

In [14]:
for mtype, model in models.items():
    log_dir = 'results/log/homo/nc/' + mtype.lower()
    loss_checkpoint_dir = f'results/checkpoints/homo/nc/{mtype.lower()}/loss'
    auc_checkpoint_dir = f'results/checkpoints/homo/nc/{mtype.lower()}/roc_auc'
    acc_checkpoint_dir = f'results/checkpoints/homo/nc/{mtype.lower()}/acc'

    os.makedirs(log_dir, exist_ok=True)
    os.makedirs(loss_checkpoint_dir, exist_ok=True)
    os.makedirs(auc_checkpoint_dir, exist_ok=True)
    os.makedirs(acc_checkpoint_dir, exist_ok=True)
    

    lr = 1e-3
    optim = torch.optim.Adam(model.parameters(), lr=lr)
    model.set_optimizer(optim)

    wandb_logger = WandbLogger(
        project="Homo_NodeClassification_finalv1",
        log_model=True,
        save_dir=log_dir,
        name=mtype,
        entity='ssc_project'

    )

    loss_checkpoint_callback = ModelCheckpoint(
        monitor=f'val_loss',
        dirpath=loss_checkpoint_dir,
        filename='HomoNodeClsf-{epoch:02d}-{val_loss:.2f}',
        save_top_k=3,
        save_last=True,
        mode='min',
        every_n_epochs=1
    )
    roc_auc_checkpoint_callback = ModelCheckpoint(
        monitor=f'val_roc_auc',
        dirpath=auc_checkpoint_dir,
        filename='HomoNodeClsf-{epoch:02d}-{val_roc_auc:.2f}',
        save_top_k=3,
        save_last=True,
        mode='max',
        every_n_epochs=1
    )
    acc_checkpoint_callback = ModelCheckpoint(
        monitor=f'val_accuracy',
        dirpath=acc_checkpoint_dir,
        filename='HomoNodeClsf-{epoch:02d}-{val_accuracy:.2f}',
        save_top_k=3,
        save_last=True,
        mode='max',
        every_n_epochs=1
    )

    trainer = L.Trainer(
        max_epochs=500,
        check_val_every_n_epoch=10,
        callbacks=[
            loss_checkpoint_callback, 
            roc_auc_checkpoint_callback,
            acc_checkpoint_callback,
        ],
        logger=wandb_logger,
        log_every_n_steps=1
    )

    trainer.fit(model, train_loader, val_loader)
    wandb.finish()

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX A6000') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: hontrn9122 (ssc_project). Use `wandb login --relogin` to force relogin


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.9/dist-packages/lightning/pytorch/utilities/model_summary/model_summary.py:454: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.

  | Name    | Type              | Params
----------------------------------------------
0 | encoder | Sequential        | 246 K 
1 | mlp     | Sequential        | 164 K 
2 | crit    | BCEWithLogitsLoss | 0     
----------------------------------------------
411 K     Trainable params
0         Non-trainable params
411 K     Total params
1.644     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=500` reached.


epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train_loss,█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_accuracy,█▄▆▁▇▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆
val_f1,█▄▆▁▇▆▇▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▆▇▆▆▆▆▆▆
val_loss,█▄▃▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_precision,▅▁▄▄▅▂▅▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▅▅▂▃▅█▂▃▃▃▃▁
val_recall,▄▁▃█▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_roc_auc,▃▁▃▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇███████████████████
epoch,499
train_loss,0.19547
